<a href="https://colab.research.google.com/github/Muppidathe/deep_learning/blob/main/ann_pytorch(classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np

In [130]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Churn_Modelling.csv')
df.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# data transformation

In [131]:
#drop unwanted column
df=df.drop(columns=['RowNumber','CustomerId','Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [132]:
from sklearn.preprocessing import LabelEncoder
print(df['Gender'].unique())
gender_encoder=LabelEncoder()
df['Gender']=gender_encoder.fit_transform(df['Gender'])
df.head()

['Female' 'Male']


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [133]:
from sklearn.preprocessing import OneHotEncoder
print(df['Geography'].unique())
print('len: ',len(df['Geography'].unique()))
geo_encoder=OneHotEncoder()
temp_df=geo_encoder.fit_transform(df[['Geography']]).toarray()
temp_df=pd.DataFrame(temp_df,columns=geo_encoder.get_feature_names_out())
temp_df.head()

['France' 'Spain' 'Germany']
len:  3


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [134]:


x=pd.concat([temp_df,df.drop(columns=['Geography','Exited'])],axis=1)
y=df['Exited']
x=x.astype(dtype='float32')
y=y.astype(dtype='float32')
x.shape

(10000, 12)

# checking for imbalance dataset


In [135]:
y.value_counts()

,count
Exited,
0.0,7963
1.0,2037


#  scaling all the feature using minmaxscaler

In [136]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_transformed=scaler.fit_transform(x)
x_transformed[:2,:]

array([[1.        , 0.        , 0.        , 0.538     , 0.        ,
        0.32432434, 0.2       , 0.        , 0.        , 1.        ,
        1.        , 0.5067349 ],
       [0.        , 0.        , 1.        , 0.51600003, 0.        ,
        0.31081083, 0.1       , 0.33403146, 0.        , 0.        ,
        1.        , 0.56270874]], dtype=float32)

#train test spliting

In [137]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=.2,random_state=42)
np.unique(y_train, return_counts=True)

(array([0., 1.], dtype=float32), array([6356, 1644]))

# handling imbalance dataset

In [138]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto", random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)
x_train_resampled.shape

(12712, 12)

In [139]:
class mymodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(12,64)
    self.fc2=nn.Linear(64,16)
    self.fc3=nn.Linear(16,4)
    self.fc4=nn.Linear(4,1)
  def forward(self,x):
    x=nn.functional.relu(self.fc1(x))
    x=nn.functional.relu(self.fc2(x))
    x=nn.functional.relu(self.fc3(x))
    x=self.fc4(x)
    return x
model=mymodel()

In [155]:
from sklearn.metrics import precision_score,recall_score,accuracy_score
criterion =nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=.01,betas=(.8,.9))
batch_size=1024
epochs=100
for epoch in range(epochs):
  all_preds=[]
  y_true=[]
  for i in range(0,len(x_train_resampled),batch_size):
    x_batch=torch.tensor(x_train_resampled[i:i+batch_size,:])
    y_batch=torch.tensor(np.array(y_train_resampled[i:i+batch_size]).reshape(-1,1))
    optimizer.zero_grad()
    res=model(x_batch)
    loss=criterion(res,y_batch)
    loss.backward()
    optimizer.step()

    #eval
    props=(torch.sigmoid(res)>.5).float()
    all_preds.extend(props.cpu().numpy().flatten())
    y_true.extend(y_batch.cpu().numpy().flatten())

  #Accuracy
  Accuracy=accuracy_score(y_true,all_preds)
  #precision
  precision=precision_score(y_true,all_preds)
  #recall
  recall=recall_score(y_true,all_preds)


  print(f'Epoch [{epoch+1}/{ epochs}], Losss : {loss.item()}, Accuracy : {Accuracy} , precision : {precision}, recall : {recall}')

Epoch [1/100], Losss : 0.44632530212402344, Accuracy : 0.7386721208307111 , precision : 0.8229033631332482, recall : 0.6082441787287602
Epoch [2/100], Losss : 0.3817328214645386, Accuracy : 0.7494493392070485 , precision : 0.7176987505148977, recall : 0.8223725613593456
Epoch [3/100], Losss : 0.37898725271224976, Accuracy : 0.7615638766519823 , precision : 0.7496621114281423, recall : 0.7853996224040277
Epoch [4/100], Losss : 0.3885944187641144, Accuracy : 0.7527533039647577 , precision : 0.7351778656126482, recall : 0.7901195720578981
Epoch [5/100], Losss : 0.3779577612876892, Accuracy : 0.7668344870988043 , precision : 0.7589312977099236, recall : 0.7820956576463184
Epoch [6/100], Losss : 0.37950095534324646, Accuracy : 0.7502359974826935 , precision : 0.7289477472290197, recall : 0.7967275015733165
Epoch [7/100], Losss : 0.37965646386146545, Accuracy : 0.756843926998112 , precision : 0.7449362340585146, recall : 0.7811516677155443
Epoch [8/100], Losss : 0.3735336363315582, Accuracy 